<a href="https://colab.research.google.com/github/arunmadapathi-1609/diabetics_prediction_using_python.ipynb/blob/main/SENTENCE%20ORDER%20PREDICTION%20USING%20LOGISTIC%20REGRESSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
import os
print(os.listdir())

['.config', 'sentence-order-prediction.zip', 'sentenceorder_sample_submission.csv', 'test_features.csv', 'sentenceorder_train.csv', 'sample_data']


In [18]:
import zipfile

with zipfile.ZipFile('sentence-order-prediction.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [19]:
train_df = pd.read_csv('sentenceorder_train.csv')
test_df = pd.read_csv('test_features.csv')
submission_df = pd.read_csv('sentenceorder_sample_submission.csv')

In [20]:
train_df.head()

,sentence_1,sentence_2,label
0,What bothered me though was the fact that the ...,This is peculiar because if a documentary is g...,1
1,There are maybe 2 scenes in the entire movie w...,Instead you just see Chris Rock being himself ...,0
2,There's an extended sequence in a cave where h...,Not a bad way to spend a couple of hours if yo...,0
3,The best part was the fight between friends th...,"I really wanted to like this movie, but I coul...",0
4,Even for the cocaine laced 1980's this is a pa...,I don't understand why someone would want to w...,1


In [21]:
test_df.head()

,id,sentence_1,sentence_2
0,0,It looks like it was totally made by amateurs,The only thing this movie doesn't fail at is p...
1,1,For instance *POSSIBLE SPOILERS* they show pro...,<br /><br />The venezuelan political process i...
2,2,"I awake suddenly, aware that I'm drooling onto...",Why was I sleeping? Did I hit my head? Or acci...
3,3,Though Worth's obvious fake Russian accent got...,Not sure if Cannon's character was meant to be...
4,4,Then look at what this film does and you reall...,How could they possibly track him if he could ...


In [22]:
submission_df.head()

,id,predicted_label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [23]:
train_df.isnull().sum()

,0
sentence_1,8
sentence_2,7
label,0


In [24]:
test_df.isnull().sum()

,0
id,0
sentence_1,0
sentence_2,1


In [25]:
submission_df.isnull().sum()

,0
id,0
predicted_label,0


In [26]:
#converting the test to the lower case
train_df['sentence_1'] = train_df['sentence_1'].fillna('').str.lower()
train_df['sentence_2'] = train_df['sentence_2'].fillna('').str.lower()
test_df['sentence_1'] = test_df['sentence_1'].fillna('').str.lower()
test_df['sentence_2'] = test_df['sentence_2'].fillna('').str.lower()


In [27]:
train_df.head()

,sentence_1,sentence_2,label
0,what bothered me though was the fact that the ...,this is peculiar because if a documentary is g...,1
1,there are maybe 2 scenes in the entire movie w...,instead you just see chris rock being himself ...,0
2,there's an extended sequence in a cave where h...,not a bad way to spend a couple of hours if yo...,0
3,the best part was the fight between friends th...,"i really wanted to like this movie, but i coul...",0
4,even for the cocaine laced 1980's this is a pa...,i don't understand why someone would want to w...,1


In [28]:


import re

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

train_df['sentence_1'] = train_df['sentence_1'].apply(remove_punctuation)
train_df['sentence_2'] = train_df['sentence_2'].apply(remove_punctuation)
test_df['sentence_1'] = test_df['sentence_1'].apply(remove_punctuation)
test_df['sentence_2'] = test_df['sentence_2'].apply(remove_punctuation)


In [29]:
train_df.head()

,sentence_1,sentence_2,label
0,what bothered me though was the fact that the ...,this is peculiar because if a documentary is g...,1
1,there are maybe 2 scenes in the entire movie w...,instead you just see chris rock being himself ...,0
2,theres an extended sequence in a cave where he...,not a bad way to spend a couple of hours if yo...,0
3,the best part was the fight between friends th...,i really wanted to like this movie but i couldnt,0
4,even for the cocaine laced 1980s this is a pat...,i dont understand why someone would want to wa...,1


In [30]:
test_df.head()

,id,sentence_1,sentence_2
0,0,it looks like it was totally made by amateurs,the only thing this movie doesnt fail at is pr...
1,1,for instance possible spoilers they show proch...,br br the venezuelan political process is too ...
2,2,i awake suddenly aware that im drooling onto t...,why was i sleeping did i hit my head or accide...
3,3,though worths obvious fake russian accent got ...,not sure if cannons character was meant to be ...
4,4,then look at what this film does and you reall...,how could they possibly track him if he could ...


In [31]:
vectorizer = TfidfVectorizer()

train_text = pd.concat([train_df['sentence_1'], train_df['sentence_2']], axis=0)
vectorizer.fit(train_text)

train_sentence1_features = vectorizer.transform(train_df['sentence_1'])
train_sentence2_features = vectorizer.transform(train_df['sentence_2'])
test_sentence1_features = vectorizer.transform(test_df['sentence_1'])
test_sentence2_features = vectorizer.transform(test_df['sentence_2'])


In [36]:
x_train = np.concatenate([train_sentence1_features.toarray(), train_sentence2_features.toarray()], axis=1)
x_test = np.concatenate([test_sentence1_features.toarray(), test_sentence2_features.toarray()], axis=1)

In [37]:
x_train.shape

(6113, 42120)

In [38]:
x_test.shape

(1529, 42120)

In [44]:
model = LogisticRegression(random_state=42)
model.fit(x_train, train_df['label'])

LogisticRegression(random_state=42)

In [45]:
train_predictions = model.predict(x_train)
train_accuracy = accuracy_score(train_df['label'], train_predictions)
print("Accuracy on training data:", train_accuracy)

Accuracy on training data: 0.923605431048585


In [41]:
#we have got the 92.3% of the accuracy by these evalution

In [46]:
# Use the trained Logistic Regression model to predict the labels for the test features
predictions = model.predict(x_test)

In [50]:
print(predictions)

[1 1 0 ... 0 1 1]


In [52]:
print("First 5 predictions:", predictions[:5])

First 5 predictions: [1 1 0 0 0]


In [54]:
submission_df = pd.DataFrame({'id': test_df['id'], 'predicted_label': predictions})
submission_df.to_csv('submission.csv', index=False)
display(submission_df.head())

,id,predicted_label
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0
